<a href="https://colab.research.google.com/github/kimdain0222/kimdain0222/blob/main/%EB%8F%99%EC%A0%81%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')
import kagglehub
path = kagglehub.dataset_download("asjad99/mimiciii")

100%|██████████| 10.6M/10.6M [00:00<00:00, 28.6MB/s]

Extracting files...


In [3]:
df_PATIENTS = pd.read_csv('/root/.cache/kagglehub/datasets/asjad99/mimiciii/versions/1/mimic-iii-clinical-database-demo-1.4/PATIENTS.csv')
df_CHARTEVENTS = pd.read_csv('/root/.cache/kagglehub/datasets/asjad99/mimiciii/versions/1/mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv')
df_ADMISSIONS = pd.read_csv('/root/.cache/kagglehub/datasets/asjad99/mimiciii/versions/1/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv')
df_D_ITEMS = pd.read_csv('/root/.cache/kagglehub/datasets/asjad99/mimiciii/versions/1/mimic-iii-clinical-database-demo-1.4/D_ITEMS.csv')

df_ADMISSIONS = df_ADMISSIONS[df_ADMISSIONS['subject_id'] != 10120]
df_PATIENTS = df_PATIENTS[df_PATIENTS['subject_id'] != 10120]

df_PATIENTS = df_PATIENTS.loc[:,['subject_id','gender']]

In [6]:
print(df_PATIENTS.columns)
print(df_CHARTEVENTS.columns)
print(df_ADMISSIONS.columns)

Index(['subject_id', 'gender'], dtype='object')
Index(['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'itemid', 'charttime',
       'storetime', 'cgid', 'value', 'valuenum', 'valueuom', 'warning',
       'error', 'resultstatus', 'stopped'],
      dtype='object')
Index(['row_id', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data'],
      dtype='object')


In [7]:
df_PATIENTS

,subject_id,gender
0,10006,F
1,10011,F
2,10013,F
3,10017,F
4,10019,M
...,...,...
95,44083,M
96,44154,M
97,44212,F
98,44222,M


In [8]:

df_CHARTEVENTS['value'] = pd.to_numeric(df_CHARTEVENTS['value'], errors='coerce')
df_pivot = df_CHARTEVENTS[df_CHARTEVENTS['itemid'].isin([211, 618, 646])].loc[:,['subject_id','itemid','value','charttime']]\
.pivot_table(
    index=['subject_id', 'charttime'],
    columns='itemid',
    values='value'
).reset_index()
df_pivot = df_pivot.sort_values(by=['subject_id', 'charttime'])
df_pivot = df_pivot.rename(columns={211: 'Heart_Rate', 618: 'Resp_Rate', 646: 'SpO2'})

# NAN 채우기
# 각 subject_id별 평균 구하기
subject_means = df_pivot.groupby("subject_id")[["Heart_Rate", "Resp_Rate", "SpO2"]].transform("mean")

# NaN 값을 각 subject의 평균으로 채우기
df_pivot_filled = df_pivot.copy()
df_pivot_filled[["Heart_Rate", "Resp_Rate", "SpO2"]] = df_pivot[["Heart_Rate", "Resp_Rate", "SpO2"]].fillna(subject_means)

# datetime 형식으로 변환
df_pivot_filled['charttime'] = pd.to_datetime(df_pivot_filled['charttime'])

df_ADMISSIONS_flag = df_ADMISSIONS.loc[:,['subject_id','hospital_expire_flag']]

# 조인
df_merged = pd.merge(df_pivot_filled, df_ADMISSIONS_flag, on='subject_id', how='left')

In [9]:
df_pivot

itemid,subject_id,charttime,Heart_Rate,Resp_Rate,SpO2
0,10006,2164-10-23 22:40:00,104.0,25.0,98.0
1,10006,2164-10-23 23:00:00,99.0,17.0,98.0
2,10006,2164-10-24 00:00:00,96.0,20.0,98.0
3,10006,2164-10-24 01:00:00,95.0,23.0,98.0
4,10006,2164-10-24 02:00:00,92.0,91.0,97.0
...,...,...,...,...,...
7748,10132,2123-08-24 05:00:00,107.0,NaN,95.0
7749,10132,2123-08-24 05:30:00,107.0,NaN,95.0
7750,10132,2123-08-24 06:00:00,107.0,NaN,94.0
7751,10132,2123-08-24 07:00:00,119.0,22.0,94.0


# 1.3 Sequence 만들기 (Sliding Window)

In [10]:
import numpy as np

SEQ_LEN = 10  # 예: 10개 타임스텝 사용

def create_sequences(group, seq_len=SEQ_LEN):
    X, y = [], []
    data = group[['Heart_Rate', 'Resp_Rate', 'SpO2']].values
    label = group['hospital_expire_flag'].values
    for i in range(len(group) - seq_len + 1):
        X.append(data[i:i+seq_len])
        y.append(label[i+seq_len-1])  # 마지막 시점의 label
    return X, y

X_total, y_total = [], []
for _, group in df_merged.groupby('subject_id'):
    X, y = create_sequences(group)
    X_total.extend(X)
    y_total.extend(y)

X_total = np.array(X_total)
y_total = np.array(y_total)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42, stratify=y_total)

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Tensor 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64)

# LSTM 모델
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝
        return self.sigmoid(out)

model = LSTMModel(input_dim=3, hidden_dim=64)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [13]:
EPOCHS = 1201

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 40 == 0:
      print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/1201, Loss: 0.6599
Epoch 41/1201, Loss: 0.5743
Epoch 81/1201, Loss: 0.4503
Epoch 121/1201, Loss: 0.3936
Epoch 161/1201, Loss: 0.3415
Epoch 201/1201, Loss: 0.3100
Epoch 241/1201, Loss: 0.2638
Epoch 281/1201, Loss: 0.2313
Epoch 321/1201, Loss: 0.1993
Epoch 361/1201, Loss: 0.1701
Epoch 401/1201, Loss: 0.1627
Epoch 441/1201, Loss: 0.1474
Epoch 481/1201, Loss: 0.1215
Epoch 521/1201, Loss: 0.0947
Epoch 561/1201, Loss: 0.0989
Epoch 601/1201, Loss: 0.1141
Epoch 641/1201, Loss: 0.0681
Epoch 681/1201, Loss: 0.0547
Epoch 721/1201, Loss: 0.0934
Epoch 761/1201, Loss: 0.0675
Epoch 801/1201, Loss: 0.1283
Epoch 841/1201, Loss: 0.0306
Epoch 881/1201, Loss: 0.0303
Epoch 921/1201, Loss: 0.0958
Epoch 961/1201, Loss: 0.0215
Epoch 1001/1201, Loss: 0.0349
Epoch 1041/1201, Loss: 0.0829
Epoch 1081/1201, Loss: 0.0163
Epoch 1121/1201, Loss: 0.0209
Epoch 1161/1201, Loss: 0.1320
Epoch 1201/1201, Loss: 0.0153


In [14]:
from sklearn.metrics import accuracy_score, f1_score

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).squeeze().numpy()
    y_pred_class = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred_class))
print("F1 Score:", f1_score(y_test, y_pred_class))
df_pred_lstm = pd.DataFrame({'predicted_prob': y_pred})
df_pred_lstm

Accuracy: 0.8957088538837589
F1 Score: 0.8883720930232558


,predicted_prob
0,9.999968e-01
1,9.903009e-01
2,1.000000e+00
3,2.342993e-07
4,9.386345e-01
...,...
1836,5.122986e-01
1837,1.000000e+00
1838,9.977230e-01
1839,9.997907e-01


from matplotlib import pyplot as plt
df_pred_lstm['predicted_prob'].plot(kind='hist', bins=20, title='predicted_prob')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_pred_lstm['predicted_prob'].plot(kind='line', figsize=(8, 4), title='predicted_prob')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='index', y='predicted_prob', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['predicted_prob']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('predicted_prob')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['predicted_prob'].plot(kind='line', figsize=(8, 4), title='predicted_prob')
plt.gca().spines[['top', 'right']].set_visible(False)